在应用各种复杂的模型之前，首先确定好整个系统的框架更为重要。在framework板块，我将讨论以下问题：

1. 为什么要选择5日移动平均收益作为预测目标？

2. 为什么要进行标签均衡的小批量训练？

3. 该使用什么样的损失函数？

4. 回测目标：准确率、召回率和严重错误率以及验证集实盘数据回测

一、选择预测目标

1. 择时作为主要目标

    交易无非选股与择时，选股是一种更高层次的择时，因子的有效性也不是永久的，无论是选择资产还是选择因子，都是期望选择出在未来一段时间更为强势的资产、因子。

    由于重点研究的是股指期货，所以基本上也不存在选股的问题，最多只是在IF 和 IC中间做一下选择。
    
    因此整个体系重点研究择时，通过预期资产未来一段时间的变化，给出交易建议

2. 预测价格变动，而不是预测价格本身
    
    很多学术的时间序列预测都是对时间序列本身进行建模，这样的方式在对价格进行建模上会出现严重问题：
    
    看上去预测线跟实际线走势非常好看，但是一回测收益就一塌糊涂

    价格是具有强烈单位根特点的时间序列数据，直接进行建模需要严格的时间序列处理，而深度学习处理这些问题是比较棘手的。
    
    直接利用深度学习模式对价格建模很容易出现模型表现很好，损失很小，类似于伪回归，而忽略的方向的重要性，回测一塌糊涂的现象。

    因此，必须把预测目标转向一阶差分，即价格的变动上，价格的变动不仅是平稳的，有良好的时间序列性质，而且更具有实际意义，可以直接用于交易

3. 预测未来5-10天的价格
    
    那么应该预测多久的未来变动呢？
    
    预测的时间越短，噪音占比越高，随机扰动、市场情绪都会对第价格产生巨大影响，很难区分真正的信号的作用；

    预测的时间越长，宏观因素和黑天鹅事件的影响力越大，价格的变动更可能取决与强大的无法预测的外部宏观因素；

    因此，无论太长或者太短都是不合适的。模型需要保持策略与信息的时间一致性。
    
    一个高频交易的策略，应该把分钟线、盘口情况、实时新闻加入模型的特征，而不应该关注未来半年的宏观预期；

    反过来一个长线投资的交易者也不可能关注短期的日内扰动。因此需要某种程度上的“知行合一”

    5-10日的预测长度，主要有以下三个优点：

    (1). 大幅度减少了日内扰动的影响

    (2). 5-10天基本面的变化不会特别的大，即使真的遇到了特别大的基本面变动，也有足够的时间去人为干预

    (3). 有一些时序模型，如Informer等有较强的多步预测能力，结合的日频数据进行5-10步的多步预测可以充分发挥模型潜力

In [6]:
import pandas as pd
data = pd.read_csv('data/market_state.csv', index_col='Unnamed: 0')
print(f"未来1天价格的每日收益标准差：{data['label_ic_ch_next_day'].describe()['std']:.2f}")
print(f"未来5天价格的每日收益标准差：{data['label_ic_ch_next_week'].describe()['std']/5:.2f}")
print(f"未来10天价格的每日收益标准差：{data['label_ic_ch_next_hfmonth'].describe()['std']/10:.2f}")

未来1天价格的每日收益标准差：80.35
未来5天价格的每日收益标准差：36.78
未来10天价格的每日收益标准差：24.99


可以看到，通过5日移动平均的方式，大幅减少了预测目标的标准差，等效降低了噪声的影响，更有利于提高夏普比率。

但预测窗口的长度并非可以无限提升，预测窗口越长，预测窗口到预测精度也会越低；

同时，降低标准差的作用也会越来越微弱，整体平衡下来，5-10日的预测窗口是比较合适的选择。

此外，还有一个关于多日预测的小问题：如果第一天预测未来5天是上涨的开了多仓，如果第二天方向变了怎么办？

答案是：通过滚动持仓的形式来实现预测结果的实现，例如:

Day 1 20%仓位，执行第一天的投资决策

Day 2 20%仓位，执行第二天的投资决策

...

Day 6 第一天的20%仓位到期，执行第六天的投资决策

通过这种方式，可以实现用每日的交易兑现未来5天预期，保证预测结果是可交易的，且避免出现信号方向频繁变动带来的问题。